In [79]:
import pandas as pd
import json
import plotly
import plotly.graph_objects as go
import numpy as np
from copy import copy

In [80]:
import plotly.io as pio
pio.renderers.default = 'browser'

In [81]:
def read_json_file(filepath: str, label: str) -> pd.DataFrame:
    with open(filepath) as json_file:
        data = json.load(json_file)
    new_data = {}
    row_number = 1
    for generation, values in data.items():
        for individual, individual_values in values.items():
            if "agg_fitness" in individual_values.keys():
                new_data.update({str(row_number): {"Generation": int(generation), "population_number": int(individual), 
                                           "agg_fitness": individual_values["agg_fitness"],
                                           "cost": individual_values["fitness"][3],
                                           "throughput_time": individual_values["fitness"][2],
                                           "throughput": individual_values["fitness"][0],
                                           "wip": individual_values["fitness"][1],
                                            "time": individual_values["time_stamp"]
                                          }})
                row_number += 1
    df = pd.DataFrame(new_data)
    df = df.T
    df["optimizer"] = label
    if label == 'anneal':
        df['agg_fitness'] = -1 * df['agg_fitness']
    df['agg_fitness'] = 0.1 * df['throughput'] - 1.0*df['wip'] - 0.005 * df['cost']
    return df.copy()

In [82]:
anneal = "anneal_results.json"
tabu = "tabu_results.json"
ea ="ea_results.json"

In [83]:
dfs = []
dfs.append(read_json_file(anneal, "anneal"))
dfs.append(read_json_file(ea, "ea"))
dfs.append(read_json_file(tabu, "tabu"))
df = pd.concat(dfs)

In [84]:
df.head()

,Generation,population_number,agg_fitness,cost,throughput_time,throughput,wip,time,optimizer
1,0.0,0.0,-400.905682,0.0,459.280601,714.0,472.305682,3.699530,anneal
2,0.0,1.0,-1314.575505,0.0,205.265928,105.0,1325.075505,5.740410,anneal
3,0.0,2.0,-1421.002988,40000.0,212.840650,107.0,1231.702988,7.713980,anneal
4,0.0,3.0,-1421.002988,40000.0,212.840650,107.0,1231.702988,9.883787,anneal
5,0.0,4.0,-1474.560934,35000.0,212.840650,107.0,1310.260934,11.649347,anneal


In [85]:
df = df.drop_duplicates(subset=['agg_fitness', 'cost', 'throughput', 'throughput_time', 'wip', 'optimizer'])
# df = df.loc[df["agg_fitness"] > -50000]
# df = df.loc[df["agg_fitness"] < 100000]

In [86]:
def is_pareto_efficient_simple(costs):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient]<c, axis=1)  # Keep any point with a lower cost
            is_efficient[i] = True  # And keep self
    return is_efficient

# Faster than is_pareto_efficient_simple, but less readable.
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

In [87]:
df["cost"] = df["cost"].astype(float)
df["agg_fitness"] = df["agg_fitness"].astype(float)

In [88]:
df = df.loc[df["wip"] < 150]
df = df.loc[df["throughput"] > 2500]
df = df.loc[df["throughput_time"] < 100]

In [89]:
print(len(df))

1929


In [90]:
columns = ['cost', 'throughput', 'wip']
# columns = ['throughput_time', 'throughput']
df_for_pareto = df[columns].copy()
df_for_pareto['throughput'] = -df_for_pareto['throughput']

In [91]:
is_efficient = is_pareto_efficient(df_for_pareto.values)
df['is_efficient'] = is_efficient

In [92]:
df['is_efficient'].sum()

73

In [93]:
import plotly.express as px

fig = px.scatter(df, x='time', y='agg_fitness', hover_data=['cost', 'Generation', 'population_number', 'agg_fitness'],
              color=df['cost'], color_continuous_scale=px.colors.sequential.Viridis, symbol='optimizer')
fig.show()

In [94]:
import plotly.express as px

fig = px.scatter(df, x='time', y='agg_fitness', hover_data=['cost', 'Generation', 'population_number', 'agg_fitness'],
              color=df['optimizer'], color_continuous_scale=px.colors.sequential.Viridis)
fig.show()

In [95]:
import plotly.express as px

fig = px.scatter(df, x='throughput', y='wip', hover_data=['cost', 'Generation', 'population_number', 'agg_fitness'],
              color=df['cost'], color_continuous_scale=px.colors.sequential.Viridis, symbol='optimizer')
fig.show()

In [96]:
def mesh3d(df, efficient_df):
    x = df['cost']
    y = df['throughput']
    z = df['wip']
    is_efficient = df['is_efficient'].astype(int)

    data = [go.Scatter3d(x=x, y=y, z=z,
                                       mode='markers', marker=dict(
            size=5,
            color=is_efficient,                # set color to an array/list of desired values
            # colorscale='PRGn',   # choose a colorscale
            # colorscale='Portland', 
            colorscale='Geyser',
            opacity=0.6,
        ))]

    x2 = efficient_df['cost']
    y2 = efficient_df['throughput']
    z2 = efficient_df['wip']
    intensity = efficient_df['agg_fitness']

    data.append(go.Mesh3d(x=x2, y=y2, z=z2, color='grey', intensity=intensity, colorscale="Darkmint", opacity=0.95))
    data.reverse()
    fig = go.Figure(data=data)
    camera = dict(
        up=dict(x=1, y=0, z=0.5),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.25, y=-1.25, z=1.25)
    )

    fig.update_layout(scene_camera=camera)
    fig.update_layout(
        scene = dict(
            xaxis = dict(nticks=4, range=[-1000,100000],),
            yaxis = dict(nticks=4, range=[3200,3500],),
            zaxis = dict(nticks=4, range=[0,40],),),
        # width=700,
        # margin=dict(r=20, l=10, b=10, t=10)
    )
    fig.show()

In [97]:
df_efficient = df.loc[df['is_efficient']]

mesh3d(df, df_efficient)
mesh3d(df.loc[df['optimizer'] == 'ea'], df_efficient)
mesh3d(df.loc[df['optimizer'] == 'anneal'], df_efficient)
mesh3d(df.loc[df['optimizer'] == 'tabu'], df_efficient)

In [98]:
df_efficient = df.loc[df['is_efficient']]
columns = ['agg_fitness', 'cost', 'throughput_time', 'throughput', 'wip', 'is_efficient']
df_efficient = df_efficient[columns]
df_efficient.rename(columns={'is_efficient': 'red_is_efficient'}, inplace=True)
df_efficient

,agg_fitness,cost,throughput_time,throughput,wip,red_is_efficient
1383,61.478723,55000.0,25.183302,3475.0,11.021277,True
1396,110.730507,45000.0,26.216839,3472.0,11.469493,True
1553,251.656825,10000.0,86.381684,3398.0,38.143175,True
1559,164.417297,30000.0,55.353773,3371.0,22.682703,True
1565,250.993656,10000.0,99.042031,3418.0,40.806344,True
...,...,...,...,...,...,...
2312,90.499916,50000.0,26.799177,3525.0,12.000084,True
2313,91.957628,50000.0,27.188191,3542.0,12.242372,True
2377,128.637832,40000.0,22.497850,3384.0,9.762168,True
2383,128.108659,40000.0,22.000157,3376.0,9.491341,True


In [99]:
df = pd.merge(df, df_efficient, how='left')

In [100]:
is_efficient_cum_sum = df.groupby(by='optimizer')['is_efficient'].cumsum()
df['is_efficient_cum_sum'] = is_efficient_cum_sum

In [101]:
import plotly.express as px

fig = px.scatter(df, x='time', y='is_efficient_cum_sum', hover_data=['cost', 'Generation', 'population_number', 'agg_fitness'],
              color=df['optimizer'])
fig.show()

In [102]:
dfs = []
dfs.append(read_json_file(anneal, "anneal"))
dfs.append(read_json_file(ea, "ea"))
dfs.append(read_json_file(tabu, "tabu"))
df = pd.concat(dfs)

In [103]:
df = df.loc[df["agg_fitness"] > -50000]
# df = df.loc[df['Generation'] > 10]

In [112]:
grouped = df.groupby(by=['optimizer'])
mean_values = grouped.rolling(200)['agg_fitness'].mean()
std_values = grouped.rolling(200)['agg_fitness'].std()
max_values = grouped['agg_fitness'].cummax()
max_values.index = std_values.index
time = grouped['time'].rolling(1).mean()

In [113]:
def hex_to_rgba(h, alpha):
    '''
    converts color value in hex format to rgba format with alpha transparency
    '''
    return tuple([int(h.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)] + [alpha])

In [114]:
def add_scatter(fig, time, mean_values, std_values, max_values, colors, label):
    color = colors.pop()
    fig.add_scatter(
    name=label + "mean fitness",
    x=time,
    y=mean_values,
    mode="lines",
    line=dict(color=color)
            )
    # fig.add_scatter(
    #     name=label + "upper fitness",
    #     x=time,
    #     y=mean_values - std_values,
    #     mode="lines",
    #     line=dict(dash='dash', color=color), showlegend=False
    #             )
    # fig.add_scatter(
    #     name=label + "lower fitness",
    #     x=time,
    #     y=mean_values + std_values,
    #     mode="lines",
    #     line=dict(dash='dash', color=color),
    #     fill="tonexty",
    #     fillcolor='rgba' + str(hex_to_rgba(color, 0.2)), showlegend=False
    # )
    fig.add_scatter(
        name=label + "max fitness",
        x=time,
        y=max_values,
        mode="lines",
        line=dict(color=color, dash='dash')
    )

In [115]:
fig = go.Figure()        
colors = copy(px.colors.qualitative.G10)
for optimizer in df['optimizer'].unique():
    add_scatter(fig, time[optimizer], mean_values[optimizer], std_values[optimizer], max_values[optimizer], colors, label=optimizer)
fig.show()